# Traducción Automática de Datos
Traducción de los datasets de inglés a español usando librerías de traducción automática.

In [1]:
import pandas as pd
from deep_translator import GoogleTranslator
from tqdm import tqdm

tqdm.pandas()

# 1. Leer los archivos
df_true = pd.read_csv('../data/raw/True.csv')
df_fake = pd.read_csv('../data/raw/Fake.csv')

# 2. Filtrar por subject
df_true_world = df_true[df_true['subject'] == 'worldnews'].sample(n=250, random_state=42)
df_fake_news = df_fake[df_fake['subject'] == 'News'].sample(n=250, random_state=42)

# 3. Agregar columna label
df_true_world['label'] = 'real'
df_fake_news['label'] = 'fake'

# 4. Unir y resetear índice
df = pd.concat([df_true_world, df_fake_news], ignore_index=True)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)  # Mezclar

# 5. Función de traducción segura
def traducir_texto(texto):
    try:
        if len(str(texto)) > 500:
            partes = [texto[i:i+500] for i in range(0, len(texto), 500)]
            traducido = []
            for parte in partes:
                traducido.append(GoogleTranslator(source='auto', target='es').translate(parte))
            return " ".join(traducido)
        else:
            return GoogleTranslator(source='auto', target='es').translate(texto)
    except Exception as e:
        print(f"Error traduciendo: {texto[:30]}... -> {e}")
        return texto  # Devuelve el original si falla

# 6. Traducir título y texto
df['title_es'] = df['title'].progress_apply(traducir_texto)
df['text_es'] = df['text'].progress_apply(traducir_texto)

# 7. Guardar el resultado
df[['title_es', 'text_es', 'subject', 'date', 'label']].to_csv('../data/translated/worldnews_balanced_es.csv', index=False)

print("✅ Traducción y guardado completados. Archivo: /data/translated/worldnews_balanced_es.csv")

  2%|██▏                                                                                        | 12/500 [01:21<1:17:12,  9.49s/it]

Error traduciendo: BERLIN (Reuters) - German lawm... -> ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


100%|████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [56:27<00:00,  6.78s/it]

✅ Traducción y guardado completados. Archivo: /data/translated/worldnews_balanced_es.csv
